In [1]:
import math 
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
df = pd.read_excel('oil.xlsx')
df.dropna(axis=0,inplace=True)
data_oil=df.reset_index()['Closing Value']

In [3]:
scaler=MinMaxScaler(feature_range=(0,1))
data_oil=scaler.fit_transform(np.array(data_oil).reshape(-1,1))

In [4]:
training_size=int(len(data_oil)*0.65)
test_size=len(data_oil)-training_size
train_data,test_data=data_oil[0:training_size,:],data_oil[training_size:len(data_oil),:1]

In [5]:
def create_dataset(dataset,time_step=1):
  dataX,dataY=[],[]
  for i in range(len(dataset)-time_step-1):
    a=dataset[i:(i+time_step),0]
    dataX.append(a)
    dataY.append(dataset[i+time_step,0])
  return np.array(dataX),np.array(dataY)

In [6]:
time_step=10
x_train,y_train=create_dataset(train_data,time_step)
x_test,y_test=create_dataset(test_data,time_step)

In [7]:
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)

In [8]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(10,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 50)            10400     
                                                                 
 lstm_1 (LSTM)               (None, 10, 50)            20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64,verbose=1)

Epoch 1/10
84/84 [==============================] - 8s 32ms/step - loss: 0.0018 - val_loss: 8.3768e-04
Epoch 2/10
84/84 [==============================] - 1s 16ms/step - loss: 1.3021e-04 - val_loss: 7.5094e-04
Epoch 3/10
84/84 [==============================] - 1s 15ms/step - loss: 1.2582e-04 - val_loss: 0.0011
Epoch 4/10
84/84 [==============================] - 2s 18ms/step - loss: 1.2572e-04 - val_loss: 0.0010
Epoch 5/10
84/84 [==============================] - 1s 17ms/step - loss: 1.2167e-04 - val_loss: 0.0017
Epoch 6/10
84/84 [==============================] - 1s 15ms/step - loss: 1.2130e-04 - val_loss: 7.3130e-04
Epoch 7/10
84/84 [==============================] - 1s 16ms/step - loss: 1.3909e-04 - val_loss: 0.0015
Epoch 8/10
84/84 [==============================] - 1s 16ms/step - loss: 1.2219e-04 - val_loss: 8.2255e-04
Epoch 9/10
84/84 [==============================] - 1s 16ms/step - loss: 1.2339e-04 - val_loss: 7.5160e-04
Epoch 10/10
84/84 [==============================] - 2s 1

In [10]:
train_predict=scaler.inverse_transform(train_data) 
test_predict=scaler.inverse_transform(test_data)

math.sqrt(mean_squared_error(train_data,train_predict))

29.347830443269938

In [11]:
model.save("crude_oil.h5")